In [6]:
import requests
from bs4 import BeautifulSoup
import re
import string
from collections import defaultdict

#### Это код, который находит ссылки всех статей:

In [263]:
url = 'https://risk-inform.ru/arhive.html'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
years = soup.find_all(class_='main_block')[1]
url_years = years.find_all(href=True)
years_url = []
articles_url = []
issues_url = []
for elem in url_years:
    year_href = elem['href']
    url_year = 'https://risk-inform.ru/' + year_href
    if ('arh_num' or 'board') not in url_year:
        years_url.append(url_year)
for url_year in years_url:
    ry = requests.get(url_year)
    soupy = BeautifulSoup(ry.content, 'html.parser')
    issues = soupy.find_all('span', class_='A_black')
    for iss in issues:
        iss_href = iss.find(href=True)['href']
        url_issue = 'https://risk-inform.ru/' + iss_href
        ri = requests.get(url_issue)
        soupi = BeautifulSoup(ri.content, 'html.parser')
        urls_article = soupi.find_all('li', type='square')
        for elem in urls_article:
            art_href = elem.find(href=True)['href']
            art_url = 'https://risk-inform.ru/' + art_href
            articles_url.append(art_url)
for url in articles_url:
    article(url)

AttributeError: 'NoneType' object has no attribute 'get_text'

#### Это функция, которая достает данные из статьи и кладет в файл

In [261]:
def article(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    inf = soup.find('table', style="border-bottom: 1px solid silver;").get_text()
    num = ''.join(re.findall('№[\d]+', inf))
    date = ''.join(re.findall('[\d]{1,2}\s[а-яё]+\s[\d]{4}', inf))
    article = soup.find(class_='statia')
    if article.find("p", align='right'):
        author = article.find("p", align='right').get_text()
    else:
        author = 'Автор не указан'
    if soup.find(class_='storytitle') != None:
        title = soup.find(class_='storytitle').get_text()
    elif soup.find(class_='rubrika') != None:
        title = soup.find(class_='rubrika').get_text()
        title = re.sub('\xa0\xa0\xa0\xa0', '', title)
    text = article.get_text().replace(title, '').replace(author, '').strip()
    statia = '=====\n' + url + '\n' + 'Газета «РИСК» ' + num + '\n' + date + '\n' + author + '\n' + title + '\n' + text + '\n'
    with open('placefornews.txt', 'a', encoding='utf-8') as file:
        file.write(statia)
    #data_graph = WordAfterWord(text, 4)
    #assymetry(data_graph, 3)
    pass

In [262]:
article('https://risk-inform.ru/text/27/prazd.html')


#### На этой статье я смотрела, как создается граф и как в нем ищутся нужные пары слов

In [278]:
r = requests.get('https://risk-inform.ru/article_9013.html')
soup = BeautifulSoup(r.content, 'html.parser')
inf = soup.find('table', style="border-bottom: 1px solid silver;").get_text()
num = ''.join(re.findall('№[\d]+', inf))
date = ''.join(re.findall('[\d]{2}\s[а-яё]+\s[\d]{4}', inf))
article = soup.find(class_='statia')
if article.find("p", align='right'):
    author = article.find("p", align='right').get_text()
else:
    author = 'Автор не указан'
if soup.find(class_='storytitle') != None:
    title = soup.find(class_='storytitle').get_text()
elif soup.find(class_='rubrika') != None:
    title = soup.find(class_='rubrika').get_text()
    title = re.sub('\xa0\xa0\xa0\xa0', '', title)
text = article.get_text().replace(title, '').replace(author, '')
data_graph = WordAfterWord(text, 3)
#data_graph
assymetry(data_graph, 1)

['из, переселенцев',
 'из, из',
 'с, наконец',
 'в, млн',
 'в, россии',
 'в, непростой',
 'в, но',
 'в, чём',
 'в, поддержать',
 'в, этого',
 'в, вынуждены',
 'в, туве',
 'в, больше',
 'в, парламенте',
 'в, в',
 'в, его',
 'в, и',
 'в, зачем',
 'в, эксплуатацию',
 'в, кто',
 'россии, в',
 'туве, в',
 'на, курорт',
 'на, деньги',
 'на, озеро',
 'на, на',
 'на, то',
 'и, курорт',
 'и, даже',
 'и, что',
 'и, другие',
 'и, объекты',
 'и, это',
 'и, в',
 'и, его',
 'и, нерентабельным',
 'и, и',
 'и, караоол',
 'и, нищета',
 'и, кызыла',
 'и, они',
 'и, уже',
 'и, наркоманами',
 'и, их',
 'и, путин',
 'и, тут',
 'и, шойгу',
 'и, но',
 'и, ещё',
 'и, жители',
 'комплекс, замкнутым',
 'замкнутым, комплекс',
 'курорт, сейчас',
 'курорт, на',
 'курорт, и',
 'курорт, он',
 'уже, и',
 'даже, и',
 'не, стало',
 'не, не',
 'не, но',
 'не, караоолу',
 'не, сам',
 'не, из',
 'что, и',
 'что, давно',
 'что, они',
 'будет, но',
 'сам, не',
 'млн, в',
 'чтобы, стала',
 'сейчас, курорт',
 'он, курорт',
 '

Построить граф связей слов, в котором слово представляет собой вершину, 
а взвешенная дуга показывает сколько раз слово в первой вершине упоминалось после
слова во второй вершине в окне размера N. 
Найти все пары слов, для которых наблюдается несоблюдение симметрии 
в M раз (отношение частоты дуги 1->2 к частоте дуги 2->1 больше М).

#### Это функция для создания графа

In [176]:
def WordAfterWord(text, n):
    punct = ['«', '»']
    text = text.translate(str.maketrans('', '', string.punctuation))
    for el in punct:
        text = text.replace(el, '')
    text = re.sub('\s\–\s', ' ', text)
    words = text.lower().split()
    data = defaultdict(int)
    for i in range(len(words) - (n-1)):
        word1 = words[i].lower()
        other_words = words[i:(n+i)]
        group = ' '.join(other_words)
        data[group] += 1
    data_graph = defaultdict(lambda: None)
    for word in list(data.keys()):
        word1, word2 = word.split()[0], word.split()[1]
        data_graph[word1] = defaultdict(int)
        data_graph[word2] = defaultdict(int)
    for group, num in data.items():
        ws = group.split()
        for j in range(len(ws)-1):
            data_graph[ws[0]][ws[j+1]] = num
        for k in range(len(ws)-2):
            data_graph[ws[1]][ws[k+2]] = num
    return data_graph

#### Это функция для поиска пар в графе

In [271]:
def assymetry(data_graph, m):
    assymetrical_pairs = []
    for word in data_graph:
        for word1 in data_graph[word]:
            if word1 in data_graph:
                if data_graph[word1][word] != 0:
                    if (data_graph[word][word1]/data_graph[word1][word]) >= m:
                        assymetrical_pairs.append(word + ', ' + word1)
    return assymetrical_pairs